# Asymmetric Coupling Exploration

**Author:** Jason Holt  
**Date:** 2025-12-06  
**Research Question:** How do asymmetric couplings affect cascade dynamics?

---

## Background

The earth system model uses asymmetric probability fractions from Wunderling et al.:

| Link | Strength | Reverse Link | Strength | Ratio |
|------|----------|--------------|----------|-------|
| THC → GIS | 0.578 | GIS → THC | 0.312 | 1.85x |
| WAIS → GIS | 0.193 | GIS → WAIS | 0.271 | 0.71x |
| WAIS → THC | 0.120 | THC → WAIS | 0.132 | 0.91x |
| THC → AMAZ | 0.372 | AMAZ → THC | **0** | ∞ |

**Key observation:** Amazon is a pure "sink" - it receives influence but exerts none.

### Experiments in this notebook:
1. Baseline: Original asymmetric couplings
2. Symmetric: Average bidirectional couplings
3. Amplified asymmetry: Double the asymmetry ratios
4. Amazon feedback: What if Amazon influenced other elements?

In [ ]:
import pycascades as pc
import numpy as np
import matplotlib.pyplot as plt

# Correct import paths for pycascades
from pycascades.core.tipping_element import cusp
from pycascades.earth_system.tipping_network_earth_system import tipping_network
from pycascades.earth_system.earth import linear_coupling_earth_system
from pycascades.earth_system.functions_earth_system import global_functions

In [ ]:
# Connect to Dask cluster for parallel computation
from dask.distributed import Client, as_completed
import time as time_module

try:
    client = Client('cascades-dask-scheduler:8786', timeout='10s')
    print(f"✓ Connected to Dask cluster")
    print(f"  Dashboard: http://localhost:30787")
    print(f"  Workers: {len(client.scheduler_info()['workers'])}")
    print(f"  Total cores: {sum(w['nthreads'] for w in client.scheduler_info()['workers'].values())}")
    DASK_AVAILABLE = True
except Exception as e:
    print(f"⚠ Dask not available: {e}")
    print("  Falling back to sequential execution")
    DASK_AVAILABLE = False

## Common Parameters

In [ ]:
# Simulation parameters
duration = 100000.0
t_step = 15
GMT = 2.5
base_strength = 0.5  # Using stronger coupling to see effects

# Timescales
gis_time, thc_time, wais_time, amaz_time = 98.0, 6.0, 48.0, 1.0

# Tipping thresholds
limits_gis = 2.24
limits_thc = 4.69
limits_wais = 1.42
limits_amaz = 4.10

# Conversion factor
conv_fac_gis = 19.035

# Noise parameters (Lévy for interesting dynamics)
alphas = [1.5, 1.5, 1.5, 1.5]
sigmas = [0.05, 0.05, 0.05, 0.05]  # Moderate noise on all elements

## Helper Function: Build Custom Network

In [ ]:
def build_network(coupling_matrix, strength=0.5):
    """
    Build a 4-element earth system network with custom coupling matrix.
    
    coupling_matrix: 4x4 array where [i,j] = influence of element i on element j
                     Indices: 0=GIS, 1=THC, 2=WAIS, 3=AMAZ
    """
    # Create tipping elements
    gis = cusp(a=-1/gis_time, b=1/gis_time, 
               c=(1/gis_time) * global_functions.CUSPc(0., limits_gis, GMT), x_0=0.0)
    thc = cusp(a=-1/thc_time, b=1/thc_time, 
               c=(1/thc_time) * global_functions.CUSPc(0., limits_thc, GMT), x_0=0.0)
    wais = cusp(a=-1/wais_time, b=1/wais_time, 
                c=(1/wais_time) * global_functions.CUSPc(0., limits_wais, GMT), x_0=0.0)
    amaz = cusp(a=-1/amaz_time, b=1/amaz_time, 
                c=(1/amaz_time) * global_functions.CUSPc(0., limits_amaz, GMT), x_0=0.0)
    
    elements = [gis, thc, wais, amaz]
    timescales = [gis_time, thc_time, wais_time, amaz_time]
    
    # Build network
    net = tipping_network()
    for elem in elements:
        net.add_element(elem)
    
    # Add couplings from matrix
    # Note: Some links are stabilizing (negative), some destabilizing (positive)
    # THC->GIS is stabilizing (negative sign in original code)
    signs = {
        (1, 0): -1,  # THC->GIS stabilizing
        (2, 0): 1,   # WAIS->GIS destabilizing  
        (0, 1): 1,   # GIS->THC destabilizing
        (2, 1): 1,   # WAIS->THC destabilizing
        (0, 2): 1,   # GIS->WAIS destabilizing
        (1, 2): 1,   # THC->WAIS destabilizing
        (1, 3): 1,   # THC->AMAZ destabilizing
        (3, 1): 1,   # AMAZ->THC (new, if enabled)
        (3, 0): 1,   # AMAZ->GIS (new, if enabled)
        (3, 2): 1,   # AMAZ->WAIS (new, if enabled)
    }
    
    for i in range(4):
        for j in range(4):
            if i != j and coupling_matrix[i, j] > 0:
                sign = signs.get((i, j), 1)
                coupling_strength = sign * (1/timescales[j]) * strength * coupling_matrix[i, j]
                net.add_coupling(i, j, linear_coupling_earth_system(strength=coupling_strength, x_0=-1))
    
    return net

In [ ]:
def run_simulation(net, seed=42):
    """Run simulation and return timeseries."""
    initial_state = [-1, -1, -1, -1]
    ev = pc.evolve_sde(net, initial_state)
    
    timestep = t_step / conv_fac_gis
    t_end = duration / conv_fac_gis
    
    ev.integrate(timestep, t_end, sigmas=sigmas, alphas=alphas, seed=seed)
    return ev.get_timeseries()

In [ ]:
def plot_results(timeseries, title):
    """Plot simulation results."""
    t = timeseries[0] * conv_fac_gis
    states = timeseries[1]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(t, states[:, 0], 'c-', label='GIS', alpha=0.8)
    ax.plot(t, states[:, 1], 'b-', label='THC', alpha=0.8)
    ax.plot(t, states[:, 2], 'k-', label='WAIS', alpha=0.8)
    ax.plot(t, states[:, 3], 'g-', label='AMAZ', alpha=0.8)
    
    ax.set_xlabel('Time [years]')
    ax.set_ylabel('System state f(x) [a.u.]')
    ax.set_title(title)
    ax.legend(loc='best')
    ax.set_xlim(0, duration)
    plt.tight_layout()
    return fig

---

## Experiment 1: Baseline (Original Asymmetric Couplings)

In [ ]:
# Original asymmetric coupling matrix
# Rows = FROM, Columns = TO
# [GIS, THC, WAIS, AMAZ]

baseline_matrix = np.array([
    #  GIS    THC    WAIS   AMAZ
    [0.000, 0.312, 0.271, 0.000],  # FROM GIS
    [0.578, 0.000, 0.132, 0.372],  # FROM THC
    [0.193, 0.120, 0.000, 0.000],  # FROM WAIS
    [0.000, 0.000, 0.000, 0.000],  # FROM AMAZ (no outgoing!)
])

print("Baseline Asymmetric Coupling Matrix:")
print("        GIS     THC    WAIS    AMAZ")
for i, label in enumerate(['GIS', 'THC', 'WAIS', 'AMAZ']):
    print(f"{label}  {baseline_matrix[i]}")

In [ ]:
net_baseline = build_network(baseline_matrix, strength=base_strength)
ts_baseline = run_simulation(net_baseline, seed=42)
fig1 = plot_results(ts_baseline, f"Exp 1: Baseline Asymmetric (strength={base_strength})")

---

## Experiment 2: Symmetric Couplings

What if all bidirectional links had equal strength (average of original)?

In [ ]:
# Symmetric: average of bidirectional pairs
symmetric_matrix = np.array([
    #  GIS    THC    WAIS   AMAZ
    [0.000, 0.445, 0.232, 0.186],  # FROM GIS (avg with reverse)
    [0.445, 0.000, 0.126, 0.186],  # FROM THC (avg with reverse)
    [0.232, 0.126, 0.000, 0.000],  # FROM WAIS
    [0.186, 0.186, 0.000, 0.000],  # FROM AMAZ (now has outgoing!)
])

print("Symmetric Coupling Matrix:")
print("        GIS     THC    WAIS    AMAZ")
for i, label in enumerate(['GIS', 'THC', 'WAIS', 'AMAZ']):
    print(f"{label}  {symmetric_matrix[i]}")

In [ ]:
net_symmetric = build_network(symmetric_matrix, strength=base_strength)
ts_symmetric = run_simulation(net_symmetric, seed=42)
fig2 = plot_results(ts_symmetric, f"Exp 2: Symmetric Couplings (strength={base_strength})")

---

## Experiment 3: Amplified Asymmetry

What if we double the asymmetry ratios? Stronger links get stronger, weaker get weaker.

In [ ]:
# Amplified asymmetry: exaggerate the differences
amplified_matrix = np.array([
    #  GIS    THC    WAIS   AMAZ
    [0.000, 0.156, 0.136, 0.000],  # FROM GIS (halved)
    [1.000, 0.000, 0.066, 0.744],  # FROM THC (doubled, capped at 1.0)
    [0.386, 0.240, 0.000, 0.000],  # FROM WAIS (doubled)
    [0.000, 0.000, 0.000, 0.000],  # FROM AMAZ (still no outgoing)
])

print("Amplified Asymmetry Matrix:")
print("        GIS     THC    WAIS    AMAZ")
for i, label in enumerate(['GIS', 'THC', 'WAIS', 'AMAZ']):
    print(f"{label}  {amplified_matrix[i]}")

In [ ]:
net_amplified = build_network(amplified_matrix, strength=base_strength)
ts_amplified = run_simulation(net_amplified, seed=42)
fig3 = plot_results(ts_amplified, f"Exp 3: Amplified Asymmetry (strength={base_strength})")

---

## Experiment 4: Amazon Feedback

What if Amazon deforestation affected other climate systems? This is physically plausible:
- Amazon moisture recycling affects Atlantic circulation → THC
- Amazon albedo changes affect global temperature → all elements

In [ ]:
# Amazon feedback: give Amazon outgoing connections
amazon_feedback_matrix = np.array([
    #  GIS    THC    WAIS   AMAZ
    [0.000, 0.312, 0.271, 0.000],  # FROM GIS (original)
    [0.578, 0.000, 0.132, 0.372],  # FROM THC (original)
    [0.193, 0.120, 0.000, 0.000],  # FROM WAIS (original)
    [0.100, 0.300, 0.050, 0.000],  # FROM AMAZ (NEW: affects THC strongly)
])

print("Amazon Feedback Matrix:")
print("        GIS     THC    WAIS    AMAZ")
for i, label in enumerate(['GIS', 'THC', 'WAIS', 'AMAZ']):
    print(f"{label}  {amazon_feedback_matrix[i]}")
print("\nNew links: AMAZ→GIS (0.1), AMAZ→THC (0.3), AMAZ→WAIS (0.05)")

In [ ]:
net_amazon = build_network(amazon_feedback_matrix, strength=base_strength)
ts_amazon = run_simulation(net_amazon, seed=42)
fig4 = plot_results(ts_amazon, f"Exp 4: Amazon Feedback (strength={base_strength})")

---

## Comparison Plot

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

datasets = [
    (ts_baseline, "1. Baseline Asymmetric"),
    (ts_symmetric, "2. Symmetric"),
    (ts_amplified, "3. Amplified Asymmetry"),
    (ts_amazon, "4. Amazon Feedback"),
]

for ax, (ts, title) in zip(axes.flat, datasets):
    t = ts[0] * conv_fac_gis
    states = ts[1]
    ax.plot(t, states[:, 0], 'c-', label='GIS', alpha=0.7)
    ax.plot(t, states[:, 1], 'b-', label='THC', alpha=0.7)
    ax.plot(t, states[:, 2], 'k-', label='WAIS', alpha=0.7)
    ax.plot(t, states[:, 3], 'g-', label='AMAZ', alpha=0.7)
    ax.set_title(title)
    ax.set_xlabel('Time [years]')
    ax.set_ylabel('State [a.u.]')
    ax.legend(loc='upper right', fontsize=8)
    ax.set_xlim(0, duration)

plt.suptitle(f'Asymmetric Coupling Comparison (GMT={GMT}, strength={base_strength})', fontsize=14)
plt.tight_layout()
plt.show()

---

## Analysis

### Observations:

*Run the cells above, then record your observations here:*

**Exp 1 (Baseline):**
- 

**Exp 2 (Symmetric):**
- 

**Exp 3 (Amplified):**
- 

**Exp 4 (Amazon Feedback):**
- 

### Key Questions:

1. Does symmetrizing the network change cascade behavior?
2. Does amplified asymmetry create more/less synchronization?
3. Does Amazon feedback enable cascades that weren't possible before?

---

*Next: Quantitative analysis of cascade timing and correlation.*

---

## Statistical Ensemble Analysis

Running each configuration multiple times with different random seeds to compare **distributions** of outcomes rather than single trajectories.

### Metrics we'll track:
1. **Final state** of each element (mean, std)
2. **Time spent tipped** (state > 0) for each element
3. **Number of tipping events** (crossings of x=0 threshold)
4. **Cross-correlation** between element pairs
5. **Maximum excursion** (extreme values reached)

In [ ]:
from scipy import stats
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

def extract_metrics(timeseries):
    """
    Extract key metrics from a simulation timeseries.
    
    Returns dict with metrics for each element and system-wide metrics.
    """
    t = timeseries[0] * conv_fac_gis
    states = timeseries[1]
    n_elements = states.shape[1]
    element_names = ['GIS', 'THC', 'WAIS', 'AMAZ']
    
    metrics = {}
    
    for i, name in enumerate(element_names):
        x = states[:, i]
        
        # Final state
        metrics[f'{name}_final'] = x[-1]
        
        # Mean state (excluding first 10% as burn-in)
        burn_in = len(x) // 10
        metrics[f'{name}_mean'] = np.mean(x[burn_in:])
        metrics[f'{name}_std'] = np.std(x[burn_in:])
        
        # Time spent in tipped state (x > 0)
        metrics[f'{name}_time_tipped'] = np.mean(x[burn_in:] > 0)
        
        # Number of threshold crossings (tipping events)
        crossings = np.diff(np.sign(x[burn_in:])) != 0
        metrics[f'{name}_n_crossings'] = np.sum(crossings)
        
        # Maximum excursions
        metrics[f'{name}_max'] = np.max(x[burn_in:])
        metrics[f'{name}_min'] = np.min(x[burn_in:])
    
    # Cross-correlations (at zero lag)
    burn_in = len(states) // 10
    for i in range(n_elements):
        for j in range(i+1, n_elements):
            corr = np.corrcoef(states[burn_in:, i], states[burn_in:, j])[0, 1]
            metrics[f'corr_{element_names[i]}_{element_names[j]}'] = corr
    
    return metrics


def run_ensemble(coupling_matrix, n_runs=20, strength=0.5, progress=True):
    """
    Run ensemble of simulations with different seeds.
    
    Returns list of metric dicts.
    """
    net = build_network(coupling_matrix, strength=strength)
    results = []
    
    iterator = tqdm(range(n_runs), desc="Running ensemble") if progress else range(n_runs)
    
    for seed in iterator:
        ts = run_simulation(net, seed=seed)
        metrics = extract_metrics(ts)
        metrics['seed'] = seed
        results.append(metrics)
    
    return results


def results_to_dataframe(results):
    """Convert list of metric dicts to a summary dataframe-like structure."""
    import pandas as pd
    return pd.DataFrame(results)

print("Ensemble functions defined.")

import pandas as pd

N_RUNS = 20  # Number of simulations per configuration

# Define all configurations
configurations = {
    'Baseline': baseline_matrix,
    'Symmetric': symmetric_matrix,
    'Amplified': amplified_matrix,
    'Amazon_Feedback': amazon_feedback_matrix,
}

def run_single_simulation_remote(coupling_matrix_data, strength, seed, params):
    """
    Run a single simulation on a Dask worker.
    All dependencies must be imported inside the function.
    """
    import numpy as np
    import pycascades as pc
    from pycascades.core.tipping_element import cusp
    from pycascades.earth_system.tipping_network_earth_system import tipping_network
    from pycascades.earth_system.earth import linear_coupling_earth_system
    from pycascades.earth_system.functions_earth_system import global_functions
    
    # Unpack params
    duration = params['duration']
    t_step = params['t_step']
    GMT = params['GMT']
    gis_time = params['gis_time']
    thc_time = params['thc_time']
    wais_time = params['wais_time']
    amaz_time = params['amaz_time']
    limits_gis = params['limits_gis']
    limits_thc = params['limits_thc']
    limits_wais = params['limits_wais']
    limits_amaz = params['limits_amaz']
    conv_fac_gis = params['conv_fac_gis']
    alphas = params['alphas']
    sigmas = params['sigmas']
    
    coupling_matrix = np.array(coupling_matrix_data)
    
    # Build network
    gis = cusp(a=-1/gis_time, b=1/gis_time, 
               c=(1/gis_time) * global_functions.CUSPc(0., limits_gis, GMT), x_0=0.0)
    thc = cusp(a=-1/thc_time, b=1/thc_time, 
               c=(1/thc_time) * global_functions.CUSPc(0., limits_thc, GMT), x_0=0.0)
    wais = cusp(a=-1/wais_time, b=1/wais_time, 
                c=(1/wais_time) * global_functions.CUSPc(0., limits_wais, GMT), x_0=0.0)
    amaz = cusp(a=-1/amaz_time, b=1/amaz_time, 
                c=(1/amaz_time) * global_functions.CUSPc(0., limits_amaz, GMT), x_0=0.0)
    
    elements = [gis, thc, wais, amaz]
    timescales = [gis_time, thc_time, wais_time, amaz_time]
    
    net = tipping_network()
    for elem in elements:
        net.add_element(elem)
    
    signs = {
        (1, 0): -1, (2, 0): 1, (0, 1): 1, (2, 1): 1,
        (0, 2): 1, (1, 2): 1, (1, 3): 1, (3, 1): 1, (3, 0): 1, (3, 2): 1,
    }
    
    for i in range(4):
        for j in range(4):
            if i != j and coupling_matrix[i, j] > 0:
                sign = signs.get((i, j), 1)
                coupling_strength = sign * (1/timescales[j]) * strength * coupling_matrix[i, j]
                net.add_coupling(i, j, linear_coupling_earth_system(strength=coupling_strength, x_0=-1))
    
    # Run simulation
    initial_state = [-1, -1, -1, -1]
    ev = pc.evolve_sde(net, initial_state)
    timestep = t_step / conv_fac_gis
    t_end = duration / conv_fac_gis
    ev.integrate(timestep, t_end, sigmas=sigmas, alphas=alphas, seed=seed)
    
    t, states = ev.get_timeseries()
    t = t * conv_fac_gis
    
    # Extract metrics
    element_names = ['GIS', 'THC', 'WAIS', 'AMAZ']
    metrics = {'seed': seed}
    
    for i, name in enumerate(element_names):
        x = states[:, i]
        burn_in = len(x) // 10
        metrics[f'{name}_final'] = x[-1]
        metrics[f'{name}_mean'] = np.mean(x[burn_in:])
        metrics[f'{name}_std'] = np.std(x[burn_in:])
        metrics[f'{name}_time_tipped'] = np.mean(x[burn_in:] > 0)
        crossings = np.diff(np.sign(x[burn_in:])) != 0
        metrics[f'{name}_n_crossings'] = np.sum(crossings)
        metrics[f'{name}_max'] = np.max(x[burn_in:])
        metrics[f'{name}_min'] = np.min(x[burn_in:])
    
    burn_in = len(states) // 10
    for i in range(4):
        for j in range(i+1, 4):
            corr = np.corrcoef(states[burn_in:, i], states[burn_in:, j])[0, 1]
            metrics[f'corr_{element_names[i]}_{element_names[j]}'] = corr
    
    return metrics

# Parameters to pass to workers
sim_params = {
    'duration': duration, 't_step': t_step, 'GMT': GMT,
    'gis_time': gis_time, 'thc_time': thc_time, 'wais_time': wais_time, 'amaz_time': amaz_time,
    'limits_gis': limits_gis, 'limits_thc': limits_thc, 'limits_wais': limits_wais, 'limits_amaz': limits_amaz,
    'conv_fac_gis': conv_fac_gis, 'alphas': alphas, 'sigmas': sigmas
}

# Run ensembles
ensemble_results = {}
total_sims = N_RUNS * len(configurations)

print(f"Running {N_RUNS} simulations for each of {len(configurations)} configurations...")
print(f"Total simulations: {total_sims}\n")

start_time = time_module.time()

if DASK_AVAILABLE and len(client.scheduler_info()['workers']) > 0:
    # PARALLEL EXECUTION
    n_workers = len(client.scheduler_info()['workers'])
    print(f"Using Dask parallel execution ({n_workers} workers)")
    print(f"Expected speedup: ~{n_workers}x\n")
    
    # Submit ALL jobs across all configurations
    all_futures = {}
    for name, matrix in configurations.items():
        matrix_list = matrix.tolist()  # Convert to list for serialization
        for seed in range(N_RUNS):
            future = client.submit(run_single_simulation_remote, matrix_list, base_strength, seed, sim_params)
            all_futures[future] = (name, seed)
    
    # Collect results as they complete
    results_by_config = {name: [] for name in configurations.keys()}
    completed = 0
    
    for future in as_completed(all_futures.keys()):
        config_name, seed = all_futures[future]
        try:
            result = future.result()
            results_by_config[config_name].append(result)
        except Exception as e:
            print(f"  Error in {config_name} seed {seed}: {e}")
        
        completed += 1
        if completed % 10 == 0 or completed == total_sims:
            print(f"  Completed {completed}/{total_sims} simulations...")
    
    # Convert to DataFrames
    for name in configurations.keys():
        ensemble_results[name] = pd.DataFrame(results_by_config[name])
else:
    # SEQUENTIAL FALLBACK
    print("Using sequential execution (Dask not available)")
    for name, matrix in configurations.items():
        print(f"\n=== {name} ===")
        results = run_ensemble(matrix, n_runs=N_RUNS, strength=base_strength)
        ensemble_results[name] = results_to_dataframe(results)

elapsed = time_module.time() - start_time
print(f"\n✓ All ensembles complete in {elapsed:.1f} seconds")
print(f"  Average: {elapsed/total_sims:.2f} sec/simulation")

In [ ]:
import pandas as pd

N_RUNS = 20  # Number of simulations per configuration

# Define all configurations
configurations = {
    'Baseline': baseline_matrix,
    'Symmetric': symmetric_matrix,
    'Amplified': amplified_matrix,
    'Amazon_Feedback': amazon_feedback_matrix,
}

# Run ensembles for each configuration
ensemble_results = {}

print(f"Running {N_RUNS} simulations for each of {len(configurations)} configurations...")
print(f"Total simulations: {N_RUNS * len(configurations)}\n")

for name, matrix in configurations.items():
    print(f"\n=== {name} ===")
    results = run_ensemble(matrix, n_runs=N_RUNS, strength=base_strength)
    ensemble_results[name] = results_to_dataframe(results)
    
print("\n✓ All ensembles complete!")

### Summary Statistics

Compare mean and standard deviation across configurations.

In [ ]:
# Create summary table
element_names = ['GIS', 'THC', 'WAIS', 'AMAZ']

summary_data = []
for config_name, df in ensemble_results.items():
    row = {'Configuration': config_name}
    for elem in element_names:
        row[f'{elem}_mean'] = f"{df[f'{elem}_mean'].mean():.3f} ± {df[f'{elem}_mean'].std():.3f}"
        row[f'{elem}_tipped%'] = f"{df[f'{elem}_time_tipped'].mean()*100:.1f}%"
        row[f'{elem}_crossings'] = f"{df[f'{elem}_n_crossings'].mean():.1f}"
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)

# Display key metrics
print("=" * 80)
print("MEAN STATE (after burn-in) - format: mean ± std across ensemble")
print("=" * 80)
print(summary_df[['Configuration'] + [f'{e}_mean' for e in element_names]].to_string(index=False))

print("\n" + "=" * 80)
print("TIME SPENT IN TIPPED STATE (x > 0)")
print("=" * 80)
print(summary_df[['Configuration'] + [f'{e}_tipped%' for e in element_names]].to_string(index=False))

print("\n" + "=" * 80)
print("MEAN NUMBER OF THRESHOLD CROSSINGS")
print("=" * 80)
print(summary_df[['Configuration'] + [f'{e}_crossings' for e in element_names]].to_string(index=False))

### Distribution Comparisons

Box plots comparing distributions across configurations.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

config_names = list(ensemble_results.keys())
colors = {'GIS': 'cyan', 'THC': 'blue', 'WAIS': 'black', 'AMAZ': 'green'}

# Plot 1: Mean State Distribution
ax = axes[0, 0]
positions = np.arange(len(config_names))
width = 0.2
for i, elem in enumerate(element_names):
    data = [ensemble_results[c][f'{elem}_mean'].values for c in config_names]
    bp = ax.boxplot(data, positions=positions + i*width - 0.3, widths=width, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor(colors[elem])
        patch.set_alpha(0.6)
ax.set_xticks(positions)
ax.set_xticklabels(config_names, rotation=15)
ax.set_ylabel('Mean State [a.u.]')
ax.set_title('Distribution of Mean States')
ax.axhline(y=0, color='r', linestyle='--', alpha=0.5, label='Tipping threshold')
ax.legend([plt.Rectangle((0,0),1,1, fc=colors[e], alpha=0.6) for e in element_names], 
          element_names, loc='upper right', fontsize=8)

# Plot 2: Time Spent Tipped
ax = axes[0, 1]
for i, elem in enumerate(element_names):
    data = [ensemble_results[c][f'{elem}_time_tipped'].values * 100 for c in config_names]
    bp = ax.boxplot(data, positions=positions + i*width - 0.3, widths=width, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor(colors[elem])
        patch.set_alpha(0.6)
ax.set_xticks(positions)
ax.set_xticklabels(config_names, rotation=15)
ax.set_ylabel('Time Tipped [%]')
ax.set_title('Time Spent in Tipped State (x > 0)')
ax.legend([plt.Rectangle((0,0),1,1, fc=colors[e], alpha=0.6) for e in element_names], 
          element_names, loc='upper right', fontsize=8)

# Plot 3: Number of Crossings
ax = axes[1, 0]
for i, elem in enumerate(element_names):
    data = [ensemble_results[c][f'{elem}_n_crossings'].values for c in config_names]
    bp = ax.boxplot(data, positions=positions + i*width - 0.3, widths=width, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor(colors[elem])
        patch.set_alpha(0.6)
ax.set_xticks(positions)
ax.set_xticklabels(config_names, rotation=15)
ax.set_ylabel('Number of Crossings')
ax.set_title('Threshold Crossing Events')
ax.legend([plt.Rectangle((0,0),1,1, fc=colors[e], alpha=0.6) for e in element_names], 
          element_names, loc='upper right', fontsize=8)

# Plot 4: Cross-correlations (GIS-THC, THC-AMAZ)
ax = axes[1, 1]
corr_pairs = ['corr_GIS_THC', 'corr_GIS_WAIS', 'corr_THC_AMAZ', 'corr_GIS_AMAZ']
pair_labels = ['GIS-THC', 'GIS-WAIS', 'THC-AMAZ', 'GIS-AMAZ']
pair_colors = ['purple', 'orange', 'brown', 'pink']

for i, (corr, label, color) in enumerate(zip(corr_pairs, pair_labels, pair_colors)):
    data = [ensemble_results[c][corr].values for c in config_names]
    bp = ax.boxplot(data, positions=positions + i*width - 0.3, widths=width, patch_artist=True)
    for patch in bp['boxes']:
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
ax.set_xticks(positions)
ax.set_xticklabels(config_names, rotation=15)
ax.set_ylabel('Correlation')
ax.set_title('Cross-Correlations Between Elements')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.legend([plt.Rectangle((0,0),1,1, fc=c, alpha=0.6) for c in pair_colors], 
          pair_labels, loc='upper right', fontsize=8)

plt.suptitle(f'Ensemble Statistics (N={N_RUNS} runs per configuration)', fontsize=14)
plt.tight_layout()
plt.show()

### Statistical Significance Tests

Using Mann-Whitney U test to compare distributions between configurations (non-parametric, appropriate for non-Gaussian Lévy-driven data).

In [ ]:
from scipy.stats import mannwhitneyu, kruskal

def compare_configurations(metric_name, alternative='two-sided'):
    """
    Compare a metric across all configurations using Kruskal-Wallis H-test
    (non-parametric ANOVA), then pairwise Mann-Whitney U tests.
    """
    config_names = list(ensemble_results.keys())
    
    # Get data for each config
    data = [ensemble_results[c][metric_name].values for c in config_names]
    
    # Kruskal-Wallis H-test (are any distributions different?)
    h_stat, p_kruskal = kruskal(*data)
    
    print(f"\n{'='*60}")
    print(f"Metric: {metric_name}")
    print(f"{'='*60}")
    print(f"Kruskal-Wallis H-test: H={h_stat:.3f}, p={p_kruskal:.4f}")
    
    if p_kruskal < 0.05:
        print("→ Significant difference detected (p < 0.05)")
        
        # Pairwise comparisons with Baseline
        print("\nPairwise comparisons vs Baseline:")
        baseline_data = ensemble_results['Baseline'][metric_name].values
        for config in config_names[1:]:
            other_data = ensemble_results[config][metric_name].values
            u_stat, p_val = mannwhitneyu(baseline_data, other_data, alternative=alternative)
            sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
            print(f"  Baseline vs {config}: U={u_stat:.1f}, p={p_val:.4f} {sig}")
    else:
        print("→ No significant difference detected (p >= 0.05)")
    
    return p_kruskal

# Test key metrics
print("STATISTICAL COMPARISON OF CONFIGURATIONS")
print("=" * 60)

key_metrics = [
    'GIS_mean', 'THC_mean', 'WAIS_mean', 'AMAZ_mean',
    'GIS_time_tipped', 'THC_time_tipped', 'AMAZ_time_tipped',
    'corr_GIS_THC', 'corr_THC_AMAZ'
]

significant_metrics = []
for metric in key_metrics:
    p = compare_configurations(metric)
    if p < 0.05:
        significant_metrics.append(metric)

print("\n" + "=" * 60)
print(f"SUMMARY: {len(significant_metrics)}/{len(key_metrics)} metrics show significant differences")
print("=" * 60)
if significant_metrics:
    print("Significant metrics:", significant_metrics)
else:
    print("No metrics showed statistically significant differences between configurations.")

### Ensemble Analysis Conclusions

*After running the cells above, record your observations:*

**Key Questions Answered:**

1. **Does coupling topology significantly affect outcome distributions?**
   - 

2. **Which metrics show the largest differences between configurations?**
   - 

3. **Does Amazon feedback change cascade dynamics?**
   - 

4. **Are the differences scientifically meaningful or just noise?**
   - 

---

*Interpretation notes:*
- If most metrics show p > 0.05, the coupling differences may be too subtle relative to Lévy noise
- Cross-correlations are particularly sensitive to coupling structure
- Consider reducing noise (σ) or increasing coupling strength to amplify differences